<a href="https://colab.research.google.com/github/MMBAYAR/GUZ-24_ISLE421/blob/main/%C3%87ok_Ama%C3%A7l%C4%B1_Optimizasyon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#  Gurobioy modülünü kur ve çalıştır
!pip install gurobipy
from gurobipy import*

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 33.6 MB/s eta 0:00:00


In [2]:
#  Veri
#   Amaçlar
maliyet = [40000, 32000]
atık_su = [800, 1250]
iş_kazası = [0.2, 0.45]

#  Talepler
talep = [48, 28, 100]

#  Sahalardaki kömür üretim hızları
kömür = [
          (12, 4),  #  A ve B sahalarından yüksek kalite kömür üretim hızları
          (4, 4),   #  A ve B sahalarından orta kalite kömür üretim hızları
          (10, 20)  #  A ve B sahalarından düşük kalite kömür üretim hızları
          ]

In [4]:
#  m değişkeninde bir Model() nesnesini sabitle
m = Model()

#  Karar Değişkenleri
x = m.addVars( 2, vtype = 'i' ) #  len(maliyet)

#  Kısıtlar
for i in range(3):  #  len(kömür), len(talep)
  m.addConstr(
              quicksum( kömür[i][j]*x[j] for j in x.keys() ) >= talep[i]  #  range(2), range( len( kömür[0] ) )
              )

#  Amaçlar
mal = quicksum( maliyet[j]*x[j] for j in x.keys() )
asu = quicksum( atık_su[j]*x[j] for j in x.keys() )
ika = quicksum( iş_kazası[j]*x[j] for j in x.keys() )

In [10]:
m.setObjective(
                ika,            #  mal, asu ve ika için ayrı ayrı çalıştır
                GRB.MINIMIZE
                )
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 3 rows, 2 columns and 6 nonzeros
Model fingerprint: 0xb46219bf
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [4e+00, 2e+01]
  Objective range  [2e-01, 5e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 1e+02]

Loaded MIP start from previous solve with objective 2.15

Presolve time: 0.00s
Presolved: 3 rows, 2 columns, 6 nonzeros
Variable types: 0 continuous, 2 integer (0 binary)

Root relaxation: objective 2.000000e+00, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       2.0000000    2.00000  0

In [11]:
#  Amaçlar
quicksum( maliyet[j]*x[j].x for j in x.keys() ), \
quicksum( atık_su[j]*x[j].x for j in x.keys() ), \
quicksum( iş_kazası[j]*x[j].x for j in x.keys() )

(<gurobi.LinExpr: 400000.0>, <gurobi.LinExpr: 8000.0>, <gurobi.LinExpr: 2.0>)

#  Öncelikli Hedef Programlama Çözümü ( mal > ika > asu )

In [12]:
#  m değişkeninde bir Model() nesnesini sabitle
m = Model()

#  Karar Değişkenleri
x = m.addVars( 2, vtype = 'i' ) #  len(maliyet)

#  Kısıtlar
for i in range(3):  #  len(kömür), len(talep)
  m.addConstr(
              quicksum( kömür[i][j]*x[j] for j in x.keys() ) >= talep[i]  #  range(2), range( len( kömür[0] ) )
              )

#  Amaçlar
mal = quicksum( maliyet[j]*x[j] for j in x.keys() )
asu = quicksum( atık_su[j]*x[j] for j in x.keys() )
ika = quicksum( iş_kazası[j]*x[j] for j in x.keys() )

#  Maliyet için çöz
m.setObjective(
              mal,
              GRB.MINIMIZE
              )
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 3 rows, 2 columns and 6 nonzeros
Model fingerprint: 0x273beab8
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [4e+00, 2e+01]
  Objective range  [3e+04, 4e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 1e+02]
Found heuristic solution: objective 384000.00000
Presolve time: 0.00s
Presolved: 3 rows, 2 columns, 6 nonzeros
Variable types: 0 continuous, 2 integer (0 binary)

Root relaxation: objective 2.440000e+05, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 244000.000    0    2 384000.000 244000.000  36.5%     -

In [14]:
m.addConstr(
            mal <= 248000  #  Bulduğum optimal değerin sağlanması kuralını modele ekledim
            )

#  İş kazası için çöz
m.setObjective(
              ika,
              GRB.MINIMIZE
              )
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 5 rows, 2 columns and 10 nonzeros
Model fingerprint: 0xece320a1
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [4e+00, 4e+04]
  Objective range  [2e-01, 5e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 2e+05]

Loaded MIP start from previous solve with objective 2.4

Presolve removed 5 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 2.4 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.400000000000e+00, best bound 2.400000000000e+00, gap 0.0000%


In [15]:
m.addConstr(
ika <= 2.4        #  maliyet için kısıtı önceden yazmıştık
)                 #  m objesi sabit olduğu için şimdi sadece iş kazası optimal değeri kuralını ekledim

#  Atık su için çöz
m.setObjective(
asu,
GRB.MINIMIZE
)

m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 6 rows, 2 columns and 12 nonzeros
Model fingerprint: 0x0e8f0cde
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-01, 4e+04]
  Objective range  [8e+02, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+05]

Loaded MIP start from previous solve with objective 7400

Presolve removed 6 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 7400 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.400000000000e+03, best bound 7.400000000000e+03, gap 0.0000%


#  Ağırlıklı Hedef Programlama Yaklaşımı (mal:9/22, ika:7/22, asu:6/22)

In [18]:
#  m değişkeninde bir Model() nesnesini sabitle
m = Model()

#  Karar Değişkenleri
x = m.addVars( 2, vtype = 'i' ) #  len(maliyet)
#  Sapma Değişkenli
s = m.addVars( 3, vtype = 'c' )

#  Kısıtlar
for i in range(3):  #  len(kömür), len(talep)
  m.addConstr(
              quicksum( kömür[i][j]*x[j] for j in x.keys() ) >= talep[i]  #  range(2), range( len( kömür[0] ) )
              )

#  Amaçlar
mal = quicksum( maliyet[j]*x[j] for j in x.keys() )
asu = quicksum( atık_su[j]*x[j] for j in x.keys() )
ika = quicksum( iş_kazası[j]*x[j] for j in x.keys() )
#  Sapmalar
m.addConstr(
            mal <= 248000 + s[0]
            )
m.addConstr(
            ika <= 2.0 + s[1]
            )
m.addConstr(
            asu <= 6950 + s[2]
            )

#  Genel Amaç Fonksiyonu
m.setObjective(
              ( 9/22*s[0] + 7/22*s[1] + 6/22*s[2] ),
              GRB.MINIMIZE
              )
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 6 rows, 5 columns and 15 nonzeros
Model fingerprint: 0x3b85168e
Variable types: 3 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-01, 4e+04]
  Objective range  [3e-01, 4e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+05]
Presolve time: 0.00s
Presolved: 6 rows, 5 columns, 15 nonzeros
Variable types: 0 continuous, 5 integer (0 binary)
Found heuristic solution: objective 57832.900000
Found heuristic solution: objective 34118.897727
Found heuristic solution: objective 10404.895455
Found heuristic solution: objective 3272.7750000
Found heuristic solution: objective 122.8545455

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 2 (of 2 available 

In [19]:
#  Amaçlar
quicksum( maliyet[j]*x[j].x for j in x.keys() ), \
quicksum( atık_su[j]*x[j].x for j in x.keys() ), \
quicksum( iş_kazası[j]*x[j].x for j in x.keys() )

(<gurobi.LinExpr: 248000.0>,
 <gurobi.LinExpr: 7400.0>,
 <gurobi.LinExpr: 2.4000000000000004>)

#  Çok Amaçlı Doğrusal Programala Yaklaşımı (mal:9/22, ika:7/22, asu:6/22)

In [24]:
#  m değişkeninde bir Model() nesnesini sabitle
m = Model()

#  Karar Değişkenleri
x = m.addVars( 2, vtype = 'i' ) #  len(maliyet)
#  Başarım Değişkenleri
b = m.addVars( 3, vtype = 'c' )

#  Kısıtlar
for i in range(3):  #  len(kömür), len(talep)
  m.addConstr(
              quicksum( kömür[i][j]*x[j] for j in x.keys() ) >= talep[i]  #  range(2), range( len( kömür[0] ) )
              )

#  Amaçlar
mal = quicksum( maliyet[j]*x[j] for j in x.keys() )
asu = quicksum( atık_su[j]*x[j] for j in x.keys() )
ika = quicksum( iş_kazası[j]*x[j] for j in x.keys() )
#  Başarımları
m.addConstr(
            (mal - 248000)/(248000) == b[0]
            )
m.addConstr(
            (ika - 2.0)/(2.0) == b[1]
            )
m.addConstr(
            (asu - 6950)/(6950) == b[2]
            )


#  Genel Amaç Fonksiyonu
m.setObjective(
              ( 9/22*b[0] + 7/22*b[1] + 6/22*b[2] ),
              GRB.MINIMIZE
              )
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 6 rows, 5 columns and 15 nonzeros
Model fingerprint: 0x8c25481b
Variable types: 3 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-01, 2e+01]
  Objective range  [3e-01, 4e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Presolve time: 0.00s
Presolved: 6 rows, 5 columns, 15 nonzeros
Variable types: 0 continuous, 5 integer (0 binary)
Found heuristic solution: objective 1.0811420

Root relaxation: objective 3.706012e-02, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       0.0370601    0.03706  0.00%     - 

In [25]:
#  Amaçlar
quicksum( maliyet[j]*x[j].x for j in x.keys() ), \
quicksum( atık_su[j]*x[j].x for j in x.keys() ), \
quicksum( iş_kazası[j]*x[j].x for j in x.keys() )

(<gurobi.LinExpr: 256000.0>,
 <gurobi.LinExpr: 6950.0>,
 <gurobi.LinExpr: 2.1500000000000004>)